<h1 align = "center">Gendered Word2Vec</h1>

In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
%matplotlib inline 

In [14]:
import json
import pyprind
import collections
import matplotlib.pyplot as plt
from textblob import TextBlob
import random

In [4]:
# load gender detector

from gender_detector import GenderDetector 
detector = GenderDetector('us')

In [5]:
# load all users

users = {}

with open("../yelp_dataset/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_user.json", "rb") as f:
    for line in f:
        user = json.loads(line.strip())
        users[user['user_id']] = user

In [ ]:
# detect all names' gender

name_set = set()

for user in users.values():
    name_set.add(user['name'])

bar = pyprind.ProgBar(len(name_set), width = 80)

# set mapping
gender = {}

for name in name_set:
    bar.update()
    try:
        gender[name] = detector.guess(name)
    except:
        gender[name] = 'unknown'

In [ ]:
# save to file

with open("gender.json", "wb") as f:
    json.dump(gender, f)

In [6]:
with open("gender.json", "rb") as f:
    gender = json.load(f)

In [7]:
# assign gender to users

for user in users.values():
    try:
        user['gender'] = gender[user['name']]
    except:
        user['gender'] = 'unknown'

In [8]:
reviews = []
with open("../yelp_dataset/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json", "rb") as f:
    for line in f:
        reviews.append(json.loads(line.strip()))

In [9]:
m_reviews = []
f_reviews = []
for review in reviews:
    user = users[review['user_id']]
    if user['gender'] == 'male':
        m_reviews.append(review)
    elif user['gender'] == 'female':
        f_reviews.append(review)

In [30]:
m_reviews_sample= random.sample(m_reviews, 10000)
f_reviews_sample= random.sample(f_reviews, 10000)

In [31]:
m_reviews_text = [r['text'] for r in m_reviews_sample]
f_reviews_text = [r['text'] for r in f_reviews_sample]

In [32]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [33]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [34]:
male_sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from male set"
for review in m_reviews_text:
    male_sentences += review_to_sentences(review, tokenizer)

female_sentences = []

print "Parsing sentences from female set"
for review in f_reviews_text:
    female_sentences += review_to_sentences(review, tokenizer)

In [35]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
male_model = word2vec.Word2Vec(male_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "male_300features_40minwords_10context"
model.save(model_name)

#=================================================
female_model = word2vec.Word2Vec(female_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "female_300features_40minwords_10context"
model.save(model_name)

In [45]:
word = "cheese"

In [46]:
male_model.most_similar(word)

[(u'bacon', 0.9200842380523682),
 (u'cheddar', 0.8842849731445312),
 (u'garlic', 0.8804785013198853),
 (u'tomato', 0.8768565654754639),
 (u'ham', 0.8741146326065063),
 (u'lettuce', 0.8739796876907349),
 (u'smoked', 0.8708423972129822),
 (u'onions', 0.8681818246841431),
 (u'mushrooms', 0.8677476644515991),
 (u'pastrami', 0.8674358129501343)]

In [47]:
female_model.most_similar(word)

[(u'tomato', 0.8612172603607178),
 (u'bacon', 0.8496257662773132),
 (u'goat', 0.8403007984161377),
 (u'tomatoes', 0.8379846811294556),
 (u'onions', 0.8369858264923096),
 (u'chili', 0.8320724964141846),
 (u'mac', 0.8304253220558167),
 (u'roasted', 0.8273231983184814),
 (u'grilled', 0.825092077255249),
 (u'pita', 0.8221036195755005)]